In [1]:
pip install pygame

In [1]:
import pygame
import random
import copy
from enum import Enum
import heapq

# Initialize Pygame
pygame.init()

# Constants
BOARD_SIZE = 5
CELL_SIZE = 100
MARGIN = 50
INFO_WIDTH = 300
WINDOW_WIDTH = BOARD_SIZE * CELL_SIZE + MARGIN * 2 + INFO_WIDTH
WINDOW_HEIGHT = BOARD_SIZE * CELL_SIZE + MARGIN * 2 + 150
MAX_TURNS = 30
WIN_ENERGY = 12

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
DARK_GRAY = (100, 100, 100)
RED = (220, 50, 50)
BLUE = (50, 50, 220)
GREEN = (50, 200, 50)
YELLOW = (255, 215, 0)
PURPLE = (150, 50, 200)
LIGHT_BLUE = (173, 216, 230)
LIGHT_RED = (255, 182, 193)
ORANGE = (255, 140, 0)

class OrbType(Enum):
    ENERGY = 1
    POWER = 2

class AIPersonality(Enum):
    BERSERKER = "Berserker"
    TACTICIAN = "Tactician"
    OPPORTUNIST = "Opportunist"

class ActionType(Enum):
    MOVE = 1
    ATTACK = 2
    TELEPORT = 3

# ==================== FUZZY LOGIC SYSTEM ====================
class FuzzyLogic:
    """Fuzzy Logic System for game state evaluation"""
    
    @staticmethod
    def triangular_membership(x, a, b, c):
        """Triangular membership function"""
        if x <= a or x >= c:
            return 0.0
        elif a < x <= b:
            return (x - a) / (b - a)
        else:
            return (c - x) / (c - b)
    
    @staticmethod
    def trapezoidal_membership(x, a, b, c, d):
        """Trapezoidal membership function"""
        if x <= a or x >= d:
            return 0.0
        elif a < x <= b:
            return (x - a) / (b - a)
        elif b < x <= c:
            return 1.0
        else:
            return (d - x) / (d - c)
    
    @staticmethod
    def fuzzify_health(health):
        """Fuzzify health value (0-4)"""
        return {
            'critical': FuzzyLogic.trapezoidal_membership(health, 0, 0, 1, 1.5),
            'low': FuzzyLogic.triangular_membership(health, 1, 2, 3),
            'good': FuzzyLogic.trapezoidal_membership(health, 2.5, 3, 4, 4)
        }
    
    @staticmethod
    def fuzzify_energy(energy):
        """Fuzzify energy value (0-12+)"""
        return {
            'low': FuzzyLogic.trapezoidal_membership(energy, 0, 0, 2, 4),
            'medium': FuzzyLogic.triangular_membership(energy, 3, 6, 9),
            'high': FuzzyLogic.trapezoidal_membership(energy, 8, 10, 15, 15)
        }
    
    @staticmethod
    def fuzzify_distance(distance):
        """Fuzzify distance (0-8)"""
        return {
            'very_close': FuzzyLogic.trapezoidal_membership(distance, 0, 0, 1, 2),
            'close': FuzzyLogic.triangular_membership(distance, 1, 2.5, 4),
            'far': FuzzyLogic.trapezoidal_membership(distance, 3, 5, 8, 8)
        }
    
    @staticmethod
    def fuzzify_turn_progress(turn, max_turns):
        """Fuzzify turn progress"""
        progress = turn / max_turns
        return {
            'early': FuzzyLogic.trapezoidal_membership(progress, 0, 0, 0.3, 0.5),
            'mid': FuzzyLogic.triangular_membership(progress, 0.3, 0.5, 0.7),
            'late': FuzzyLogic.trapezoidal_membership(progress, 0.6, 0.8, 1, 1)
        }
    
    @staticmethod
    def defuzzify_aggression(fuzzy_output):
        """Convert fuzzy aggression to crisp value using centroid method"""
        # Aggression levels: defensive(0-0.3), neutral(0.3-0.7), aggressive(0.7-1.0)
        numerator = 0
        denominator = 0
        
        # Sample points for centroid calculation
        for i in range(101):
            x = i / 100.0
            if x < 0.3:
                level = 'defensive'
                membership = (0.3 - x) / 0.3
            elif x < 0.7:
                level = 'neutral'
                membership = 1.0 - abs(x - 0.5) / 0.2
            else:
                level = 'aggressive'
                membership = (x - 0.7) / 0.3
            
            final_membership = min(membership, fuzzy_output.get(level, 0))
            numerator += x * final_membership
            denominator += final_membership
        
        return numerator / denominator if denominator > 0 else 0.5
    
    @staticmethod
    def evaluate_situation(player_health, player_energy, opponent_health, 
                          opponent_energy, distance, turn, max_turns, personality):
        """Main fuzzy inference system"""
        # Fuzzification
        health_fuzzy = FuzzyLogic.fuzzify_health(player_health)
        energy_fuzzy = FuzzyLogic.fuzzify_energy(player_energy)
        opp_health_fuzzy = FuzzyLogic.fuzzify_health(opponent_health)
        distance_fuzzy = FuzzyLogic.fuzzify_distance(distance)
        turn_fuzzy = FuzzyLogic.fuzzify_turn_progress(turn, max_turns)
        
        # Fuzzy Rules
        rules_output = {
            'defensive': 0,
            'neutral': 0,
            'aggressive': 0
        }
        
        # Rule 1: If health is critical, be defensive
        rules_output['defensive'] = max(rules_output['defensive'], 
                                       health_fuzzy['critical'])
        
        # Rule 2: If health is good and opponent health is low, be aggressive
        rules_output['aggressive'] = max(rules_output['aggressive'],
                                        min(health_fuzzy['good'], 
                                            opp_health_fuzzy['low']))
        
        # Rule 3: If distance is very close and health is good, be aggressive
        rules_output['aggressive'] = max(rules_output['aggressive'],
                                        min(distance_fuzzy['very_close'],
                                            health_fuzzy['good']))
        
        # Rule 4: If energy is high and turn is late, be aggressive
        rules_output['aggressive'] = max(rules_output['aggressive'],
                                        min(energy_fuzzy['high'],
                                            turn_fuzzy['late']))
        
        # Rule 5: If health is low and distance is close, be defensive
        rules_output['defensive'] = max(rules_output['defensive'],
                                       min(health_fuzzy['low'],
                                           distance_fuzzy['very_close']))
        
        # Rule 6: Normal situation - neutral
        rules_output['neutral'] = min(health_fuzzy['good'],
                                     energy_fuzzy['medium'])
        
        # Personality modifiers
        if personality == AIPersonality.BERSERKER:
            rules_output['aggressive'] *= 1.5
            rules_output['defensive'] *= 0.5
        elif personality == AIPersonality.TACTICIAN:
            rules_output['neutral'] *= 1.3
        elif personality == AIPersonality.OPPORTUNIST:
            if energy_fuzzy['high'] > 0.5:
                rules_output['aggressive'] *= 1.2
        
        # Normalize
        total = sum(rules_output.values())
        if total > 0:
            rules_output = {k: v/total for k, v in rules_output.items()}
        
        return FuzzyLogic.defuzzify_aggression(rules_output), rules_output


# ==================== A* PATHFINDING ====================
class AStarPathfinder:
    """A* Algorithm for optimal pathfinding"""
    
    @staticmethod
    def heuristic(x1, y1, x2, y2):
        """Manhattan distance heuristic"""
        return abs(x1 - x2) + abs(y1 - y2)
    
    @staticmethod
    def get_neighbors(x, y, board_size):
        """Get valid neighboring cells"""
        neighbors = []
        for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < board_size and 0 <= ny < board_size:
                neighbors.append((nx, ny))
        return neighbors
    
    @staticmethod
    def find_path(start_x, start_y, goal_x, goal_y, board_size, blocked_pos=None):
        """
        Find optimal path using A* algorithm
        Returns: list of (x, y) positions from start to goal
        """
        if blocked_pos is None:
            blocked_pos = set()
        
        if (goal_x, goal_y) in blocked_pos:
            return []
        
        # Priority queue: (f_score, counter, current_pos, path)
        counter = 0
        start = (start_x, start_y)
        goal = (goal_x, goal_y)
        
        open_set = [(0, counter, start, [start])]
        closed_set = set()
        g_scores = {start: 0}
        
        while open_set:
            f_score, _, current, path = heapq.heappop(open_set)
            
            if current == goal:
                return path
            
            if current in closed_set:
                continue
            
            closed_set.add(current)
            
            for neighbor in AStarPathfinder.get_neighbors(current[0], current[1], board_size):
                if neighbor in closed_set or neighbor in blocked_pos:
                    continue
                
                tentative_g = g_scores[current] + 1
                
                if neighbor not in g_scores or tentative_g < g_scores[neighbor]:
                    g_scores[neighbor] = tentative_g
                    h_score = AStarPathfinder.heuristic(neighbor[0], neighbor[1], 
                                                       goal_x, goal_y)
                    f_score = tentative_g + h_score
                    
                    counter += 1
                    new_path = path + [neighbor]
                    heapq.heappush(open_set, (f_score, counter, neighbor, new_path))
        
        return []  # No path found
    
    @staticmethod
    def find_optimal_orb(player_x, player_y, orbs, board_size, blocked_pos=None):
        """Find the closest orb using A* and return best target"""
        best_orb = None
        best_path = None
        best_score = float('inf')
        
        for orb in orbs:
            path = AStarPathfinder.find_path(player_x, player_y, orb.x, orb.y, 
                                            board_size, blocked_pos)
            if path:
                # Score based on path length and orb type
                path_length = len(path)
                orb_value = 2 if orb.type == OrbType.POWER else 1
                score = path_length / orb_value
                
                if score < best_score:
                    best_score = score
                    best_orb = orb
                    best_path = path
        
        return best_orb, best_path


# ==================== GAME CLASSES ====================
class Player:
    def __init__(self, x, y, color, name, personality):
        self.x = x
        self.y = y
        self.prev_x = x
        self.prev_y = y
        self.health = 4
        self.energy = 0
        self.color = color
        self.name = name
        self.personality = personality
        self.animation_progress = 0

class Orb:
    def __init__(self, x, y, orb_type):
        self.x = x
        self.y = y
        self.type = orb_type

class GameState:
    def __init__(self, player1, player2, orbs, turn, game_log):
        self.player1 = copy.deepcopy(player1)
        self.player2 = copy.deepcopy(player2)
        self.orbs = copy.deepcopy(orbs)
        self.turn = turn
        self.game_log = game_log.copy()

class Button:
    def __init__(self, x, y, width, height, text, color, text_color=WHITE):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.text_color = text_color
        self.hovered = False

    def draw(self, screen, font):
        color = tuple(min(c + 30, 255) for c in self.color) if self.hovered else self.color
        pygame.draw.rect(screen, color, self.rect, border_radius=5)
        pygame.draw.rect(screen, BLACK, self.rect, 2, border_radius=5)
        
        text_surf = font.render(self.text, True, self.text_color)
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

    def update_hover(self, pos):
        self.hovered = self.rect.collidepoint(pos)


# ==================== MAIN GAME CLASS ====================
class MindDuelGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Mind Duel - Fuzzy Logic & A* AI")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 28)
        self.title_font = pygame.font.Font(None, 48)
        self.small_font = pygame.font.Font(None, 22)
        
        self.game_state = "menu"
        self.paused = False
        self.turn = 0
        self.player1 = None
        self.player2 = None
        self.orbs = []
        self.game_log = []
        self.winner = None
        self.auto_play_delay = 1000
        self.last_action_time = 0
        self.current_player_turn = 1
        
        self.history = []
        self.current_history_index = -1
        
        self.setup_buttons()

    def setup_buttons(self):
        button_y = 300
        button_spacing = 80
        
        self.berserker_vs_tactician_btn = Button(
            WINDOW_WIDTH // 2 - 150, button_y, 300, 50,
            "Berserker vs Tactician", BLUE
        )
        
        self.tactician_vs_opportunist_btn = Button(
            WINDOW_WIDTH // 2 - 150, button_y + button_spacing, 300, 50,
            "Tactician vs Opportunist", GREEN
        )
        
        self.berserker_vs_opportunist_btn = Button(
            WINDOW_WIDTH // 2 - 150, button_y + button_spacing * 2, 300, 50,
            "Berserker vs Opportunist", PURPLE
        )
        
        control_y = WINDOW_HEIGHT - 120
        self.pause_btn = Button(MARGIN + 10, control_y, 100, 40, "Pause", GRAY, BLACK)
        self.backward_btn = Button(MARGIN + 120, control_y, 100, 40, "< Back", ORANGE, BLACK)
        self.forward_btn = Button(MARGIN + 230, control_y, 100, 40, "Forward >", GREEN, BLACK)
        self.restart_btn = Button(MARGIN + 340, control_y, 100, 40, "Restart", BLUE, BLACK)
        
        self.resume_btn = Button(WINDOW_WIDTH // 2 - 75, WINDOW_HEIGHT // 2 - 50, 150, 50, "Resume", GREEN)
        self.menu_btn = Button(WINDOW_WIDTH // 2 - 75, WINDOW_HEIGHT // 2 + 20, 150, 50, "Menu", BLUE)

    def save_state(self):
        state = GameState(self.player1, self.player2, self.orbs, self.turn, self.game_log)
        if self.current_history_index < len(self.history) - 1:
            self.history = self.history[:self.current_history_index + 1]
        
        self.history.append(state)
        self.current_history_index = len(self.history) - 1

    def load_state(self, state):
        self.player1 = copy.deepcopy(state.player1)
        self.player2 = copy.deepcopy(state.player2)
        self.orbs = copy.deepcopy(state.orbs)
        self.turn = state.turn
        self.game_log = state.game_log.copy()
        
        self.player1.animation_progress = 1.0
        self.player1.prev_x = self.player1.x
        self.player1.prev_y = self.player1.y
        self.player2.animation_progress = 1.0
        self.player2.prev_x = self.player2.x
        self.player2.prev_y = self.player2.y
        
        self.check_win_condition()

    def go_backward(self):
        if self.current_history_index > 0:
            self.current_history_index -= 1
            self.load_state(self.history[self.current_history_index])

    def go_forward(self):
        if self.current_history_index < len(self.history) - 1:
            self.current_history_index += 1
            self.load_state(self.history[self.current_history_index])

    def start_game(self, ai1_personality, ai2_personality):
        self.game_state = "playing"
        self.paused = False
        self.turn = 0
        self.winner = None
        self.current_player_turn = 1
        
        self.player1 = Player(0, 0, RED, "AI 1", ai1_personality)
        self.player2 = Player(4, 4, BLUE, "AI 2", ai2_personality)
        
        self.orbs = self.spawn_initial_orbs()
        self.game_log = [f"Game started: {ai1_personality.value} vs {ai2_personality.value}"]
        self.last_action_time = pygame.time.get_ticks()
        
        self.history = []
        self.current_history_index = -1
        self.save_state()

    def spawn_initial_orbs(self):
        orbs = []
        occupied = {(0, 0), (4, 4)}
        
        for _ in range(3):
            while True:
                x = random.randint(0, BOARD_SIZE - 1)
                y = random.randint(0, BOARD_SIZE - 1)
                if (x, y) not in occupied:
                    occupied.add((x, y))
                    orb_type = OrbType.ENERGY if random.random() < 0.7 else OrbType.POWER
                    orbs.append(Orb(x, y, orb_type))
                    break
        
        return orbs

    def spawn_new_orb(self):
        occupied = {(self.player1.x, self.player1.y), (self.player2.x, self.player2.y)}
        occupied.update((orb.x, orb.y) for orb in self.orbs)
        
        if len(occupied) >= BOARD_SIZE * BOARD_SIZE:
            return
        
        attempts = 0
        while attempts < 50:
            x = random.randint(0, BOARD_SIZE - 1)
            y = random.randint(0, BOARD_SIZE - 1)
            if (x, y) not in occupied:
                orb_type = OrbType.ENERGY if random.random() < 0.7 else OrbType.POWER
                self.orbs.append(Orb(x, y, orb_type))
                break
            attempts += 1

    def distance(self, x1, y1, x2, y2):
        return abs(x1 - x2) + abs(y1 - y2)

    def get_available_actions(self, player, opponent):
        actions = []
        
        moves = [(0, -1), (0, 1), (-1, 0), (1, 0)]
        
        for dx, dy in moves:
            nx, ny = player.x + dx, player.y + dy
            if 0 <= nx < BOARD_SIZE and 0 <= ny < BOARD_SIZE:
                if nx == opponent.x and ny == opponent.y:
                    actions.append({'type': ActionType.ATTACK, 'x': nx, 'y': ny})
                else:
                    actions.append({'type': ActionType.MOVE, 'x': nx, 'y': ny})
        
        if player.energy >= 4:
            for x in range(BOARD_SIZE):
                for y in range(BOARD_SIZE):
                    if (x, y) != (player.x, player.y) and (x, y) != (opponent.x, opponent.y):
                        actions.append({'type': ActionType.TELEPORT, 'x': x, 'y': y})
        
        return actions

    def evaluate_action_with_fuzzy_astar(self, action, player, opponent):
        """Enhanced evaluation using Fuzzy Logic and A*"""
        score = 0
        
        # Calculate fuzzy aggression level
        dist_to_opponent = self.distance(action['x'], action['y'], opponent.x, opponent.y)
        aggression, fuzzy_details = FuzzyLogic.evaluate_situation(
            player.health, player.energy,
            opponent.health, opponent.energy,
            dist_to_opponent, self.turn, MAX_TURNS,
            player.personality
        )
        
        # Use A* to find optimal path to nearest orb from action position
        blocked = {(opponent.x, opponent.y)}
        target_orb, path_to_orb = AStarPathfinder.find_optimal_orb(
            action['x'], action['y'], self.orbs, BOARD_SIZE, blocked
        )
        
        # Base scoring
        orb_at_target = next((o for o in self.orbs if o.x == action['x'] and o.y == action['y']), None)
        if orb_at_target:
            score += 10 if orb_at_target.type == OrbType.POWER else 5
        
        # Fuzzy-based scoring
        if action['type'] == ActionType.ATTACK:
            score += aggression * 15  # More aggressive = higher attack score
        
        if action['type'] == ActionType.MOVE:
            # Use A* path length to evaluate move quality
            if path_to_orb and len(path_to_orb) > 1:
                score += 8 / len(path_to_orb)  # Shorter path = higher score
        
        # Defensive behavior when fuzzy says so
        if aggression < 0.3:  # Defensive
            if dist_to_opponent > 2:
                score += 5
            if action['type'] == ActionType.TELEPORT:
                score += 8
        
        # Aggressive behavior
        elif aggression > 0.7:  # Aggressive
            score -= dist_to_opponent * 2
            if action['type'] == ActionType.ATTACK:
                score += 12
        
        # Personality modifiers
        if player.personality == AIPersonality.BERSERKER:
            if action['type'] == ActionType.ATTACK:
                score += 8
            if player.health <= 1 and dist_to_opponent > 3:
                score += 10
                
        elif player.personality == AIPersonality.TACTICIAN:
            if target_orb and target_orb.type == OrbType.POWER:
                score += 7
            if action['type'] == ActionType.ATTACK and opponent.health <= 2:
                score += 10
                
        elif player.personality == AIPersonality.OPPORTUNIST:
            if orb_at_target and orb_at_target.type == OrbType.POWER:
                score += 12
            if player.energy > opponent.energy and action['type'] == ActionType.ATTACK:
                score += 6
        
        # Random factor for variety
        score += random.uniform(-0.5, 0.5)
        
        return score

    def choose_ai_action(self, player, opponent):
        actions = self.get_available_actions(player, opponent)
        if not actions:
            return None
        
        best_action = max(actions, key=lambda a: self.evaluate_action_with_fuzzy_astar(a, player, opponent))
        return best_action

    def execute_action(self, player, action, opponent):
        player.prev_x = player.x
        player.prev_y = player.y
        player.animation_progress = 0
        
        if action['type'] == ActionType.ATTACK:
            opponent.health -= 1
            player.x, player.y = action['x'], action['y']
            self.game_log.append(f"{player.name} attacks {opponent.name}!")
            
        elif action['type'] == ActionType.TELEPORT:
            player.energy -= 4
            player.x, player.y = action['x'], action['y']
            self.game_log.append(f"{player.name} teleports!")
            
        elif action['type'] == ActionType.MOVE:
            player.x, player.y = action['x'], action['y']
        
        orb_collected = None
        for orb in self.orbs:
            if orb.x == player.x and orb.y == player.y:
                orb_collected = orb
                energy_gain = 4 if orb.type == OrbType.POWER else 2
                player.energy += energy_gain
                self.game_log.append(f"{player.name} collected {orb.type.name} orb (+{energy_gain} energy)!")
                break
        
        if orb_collected:
            self.orbs.remove(orb_collected)
            self.spawn_new_orb()

    def process_turn(self):
        if self.paused or self.winner:
            return
        
        if self.current_history_index < len(self.history) - 1:
            return
        
        current_time = pygame.time.get_ticks()
        if current_time - self.last_action_time < self.auto_play_delay:
            return
        
        self.last_action_time = current_time
        
        if self.current_player_turn == 1:
            action = self.choose_ai_action(self.player1, self.player2)
            if action:
                self.execute_action(self.player1, action, self.player2)
            self.current_player_turn = 2
        else:
            action = self.choose_ai_action(self.player2, self.player1)
            if action:
                self.execute_action(self.player2, action, self.player1)
            self.current_player_turn = 1
            self.turn += 1
        
        self.check_win_condition()
        self.save_state()
        
        if len(self.game_log) > 15:
            self.game_log.pop(0)

    def check_win_condition(self):
        if self.player1.health <= 0:
            self.winner = self.player2
            if not any("wins by knockout" in log for log in self.game_log):
                self.game_log.append(f"{self.player2.name} wins by knockout!")
        elif self.player2.health <= 0:
            self.winner = self.player1
            if not any("wins by knockout" in log for log in self.game_log):
                self.game_log.append(f"{self.player1.name} wins by knockout!")
        elif self.player1.energy >= WIN_ENERGY:
            self.winner = self.player1
            if not any(f"wins with {WIN_ENERGY} energy" in log for log in self.game_log):
                self.game_log.append(f"{self.player1.name} wins with {WIN_ENERGY} energy!")
        elif self.player2.energy >= WIN_ENERGY:
            self.winner = self.player2
            if not any(f"wins with {WIN_ENERGY} energy" in log for log in self.game_log):
                self.game_log.append(f"{self.player2.name} wins with {WIN_ENERGY} energy!")
        elif self.turn >= MAX_TURNS:
            total1 = self.player1.health + self.player1.energy
            total2 = self.player2.health + self.player2.energy
            if total1 > total2:
                self.winner = self.player1
                if not any("wins on points" in log for log in self.game_log):
                    self.game_log.append(f"{self.player1.name} wins on points!")
            elif total2 > total1:
                self.winner = self.player2
                if not any("wins on points" in log for log in self.game_log):
                    self.game_log.append(f"{self.player2.name} wins on points!")
            else:
                self.winner = "draw"
                if not any("draw" in log for log in self.game_log):
                    self.game_log.append("Game ends in a draw!")

    def draw_board(self):
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                rect = pygame.Rect(
                    MARGIN + x * CELL_SIZE,
                    MARGIN + y * CELL_SIZE,
                    CELL_SIZE,
                    CELL_SIZE
                )
                pygame.draw.rect(self.screen, GRAY, rect)
                pygame.draw.rect(self.screen, BLACK, rect, 2)

    def draw_orbs(self):
        for orb in self.orbs:
            center_x = MARGIN + orb.x * CELL_SIZE + CELL_SIZE // 2
            center_y = MARGIN + orb.y * CELL_SIZE + CELL_SIZE // 2
            
            if orb.type == OrbType.POWER:
                pygame.draw.circle(self.screen, PURPLE, (center_x, center_y), 20)
                pygame.draw.circle(self.screen, YELLOW, (center_x, center_y), 15)
            else:
                pygame.draw.circle(self.screen, YELLOW, (center_x, center_y), 15)
            
            pygame.draw.circle(self.screen, BLACK, (center_x, center_y), 15 if orb.type == OrbType.ENERGY else 20, 2)

    def draw_players(self):
        for player in [self.player1, self.player2]:
            if player.animation_progress < 1.0:
                player.animation_progress = min(1.0, player.animation_progress + 0.08)
            
            if player.animation_progress < 1.0:
                current_x = player.prev_x + (player.x - player.prev_x) * player.animation_progress
                current_y = player.prev_y + (player.y - player.prev_y) * player.animation_progress
            else:
                current_x = player.x
                current_y = player.y
            
            center_x = MARGIN + int(current_x * CELL_SIZE + CELL_SIZE // 2)
            center_y = MARGIN + int(current_y * CELL_SIZE + CELL_SIZE // 2)
            
            if player.animation_progress < 1.0 and (player.prev_x != player.x or player.prev_y != player.y):
                prev_center_x = MARGIN + player.prev_x * CELL_SIZE + CELL_SIZE // 2
                prev_center_y = MARGIN + player.prev_y * CELL_SIZE + CELL_SIZE // 2
                target_center_x = MARGIN + player.x * CELL_SIZE + CELL_SIZE // 2
                target_center_y = MARGIN + player.y * CELL_SIZE + CELL_SIZE // 2
                
                steps = 10
                for i in range(steps):
                    t = i / steps
                    dot_x = int(prev_center_x + (target_center_x - prev_center_x) * t)
                    dot_y = int(prev_center_y + (target_center_y - prev_center_y) * t)
                    dot_size = int(6 - 3 * (1 - t))
                    trail_color = tuple(min(255, c + 80) for c in player.color)
                    pygame.draw.circle(self.screen, trail_color, (dot_x, dot_y), dot_size)
                
                arrow_size = 12
                dx = target_center_x - prev_center_x
                dy = target_center_y - prev_center_y
                length = max(1, (dx**2 + dy**2)**0.5)
                dx, dy = dx/length, dy/length
                
                arrow_base_x = target_center_x - dx * 15
                arrow_base_y = target_center_y - dy * 15
                left_x = arrow_base_x - dy * arrow_size
                left_y = arrow_base_y + dx * arrow_size
                right_x = arrow_base_x + dy * arrow_size
                right_y = arrow_base_y - dx * arrow_size
                
                pygame.draw.polygon(self.screen, player.color, [
                    (target_center_x, target_center_y),
                    (int(left_x), int(left_y)),
                    (int(right_x), int(right_y))
                ])
            
            pygame.draw.circle(self.screen, player.color, (center_x, center_y), 30)
            pygame.draw.circle(self.screen, BLACK, (center_x, center_y), 30, 3)
            
            health_width = 60
            health_x = center_x - health_width // 2
            health_y = center_y - 50
            
            pygame.draw.rect(self.screen, DARK_GRAY, (health_x, health_y, health_width, 8))
            health_filled = int(health_width * (player.health / 4))
            pygame.draw.rect(self.screen, GREEN, (health_x, health_y, health_filled, 8))

    def draw_info_panel(self):
        panel_x = BOARD_SIZE * CELL_SIZE + MARGIN * 2
        
        title = self.font.render("Mind Duel", True, BLACK)
        self.screen.blit(title, (panel_x + 10, 20))
        
        subtitle = self.small_font.render("Fuzzy + A*", True, PURPLE)
        self.screen.blit(subtitle, (panel_x + 10, 45))
        
        turn_text = self.small_font.render(f"Turn: {self.turn}/{MAX_TURNS}", True, BLACK)
        self.screen.blit(turn_text, (panel_x + 10, 70))
        
        if not self.winner:
            turn_indicator = self.small_font.render(
                f"Current: {'AI 1' if self.current_player_turn == 1 else 'AI 2'}", 
                True, 
                RED if self.current_player_turn == 1 else BLUE
            )
            self.screen.blit(turn_indicator, (panel_x + 10, 95))
        
        if self.current_history_index < len(self.history) - 1:
            history_text = self.small_font.render(f"Viewing: {self.current_history_index + 1}/{len(self.history)}", True, ORANGE)
            self.screen.blit(history_text, (panel_x + 10, 120))
        
        y_offset = 150
        p1_name = self.font.render(f"{self.player1.name}", True, self.player1.color)
        self.screen.blit(p1_name, (panel_x + 10, y_offset))
        
        p1_personality = self.small_font.render(f"({self.player1.personality.value})", True, BLACK)
        self.screen.blit(p1_personality, (panel_x + 10, y_offset + 25))
        
        p1_health = self.small_font.render(f"Health: {self.player1.health}/4", True, BLACK)
        self.screen.blit(p1_health, (panel_x + 10, y_offset + 50))
        
        p1_energy = self.small_font.render(f"Energy: {self.player1.energy}", True, BLACK)
        self.screen.blit(p1_energy, (panel_x + 10, y_offset + 75))
        
        y_offset = 270
        p2_name = self.font.render(f"{self.player2.name}", True, self.player2.color)
        self.screen.blit(p2_name, (panel_x + 10, y_offset))
        
        p2_personality = self.small_font.render(f"({self.player2.personality.value})", True, BLACK)
        self.screen.blit(p2_personality, (panel_x + 10, y_offset + 25))
        
        p2_health = self.small_font.render(f"Health: {self.player2.health}/4", True, BLACK)
        self.screen.blit(p2_health, (panel_x + 10, y_offset + 50))
        
        p2_energy = self.small_font.render(f"Energy: {self.player2.energy}", True, BLACK)
        self.screen.blit(p2_energy, (panel_x + 10, y_offset + 75))
        
        log_y = 390
        log_title = self.font.render("Game Log:", True, BLACK)
        self.screen.blit(log_title, (panel_x + 10, log_y))
        
        for i, log in enumerate(self.game_log[-8:]):
            log_text = self.small_font.render(log[:30], True, DARK_GRAY)
            self.screen.blit(log_text, (panel_x + 10, log_y + 30 + i * 20))

    def draw_menu(self):
        title = self.title_font.render("Mind Duel", True, BLACK)
        title_rect = title.get_rect(center=(WINDOW_WIDTH // 2, 150))
        self.screen.blit(title, title_rect)
        
        subtitle = self.font.render("AI vs AI Battle", True, DARK_GRAY)
        subtitle_rect = subtitle.get_rect(center=(WINDOW_WIDTH // 2, 210))
        self.screen.blit(subtitle, subtitle_rect)
        
        tech_subtitle = self.small_font.render("Please Select Mode!", True, PURPLE)
        tech_rect = tech_subtitle.get_rect(center=(WINDOW_WIDTH // 2, 240))
        self.screen.blit(tech_subtitle, tech_rect)
        
        self.berserker_vs_tactician_btn.draw(self.screen, self.font)
        self.tactician_vs_opportunist_btn.draw(self.screen, self.font)
        self.berserker_vs_opportunist_btn.draw(self.screen, self.font)

    def draw_pause_overlay(self):
        overlay = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT))
        overlay.set_alpha(180)
        overlay.fill(WHITE)
        self.screen.blit(overlay, (0, 0))
        
        pause_text = self.title_font.render("PAUSED", True, BLACK)
        pause_rect = pause_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 100))
        self.screen.blit(pause_text, pause_rect)
        
        self.resume_btn.draw(self.screen, self.font)
        self.menu_btn.draw(self.screen, self.font)

    def draw_winner(self):
        if self.winner == "draw":
            text = "Draw!"
        else:
            text = f"{self.winner.name} Wins!"
        
        winner_text = self.title_font.render(text, True, BLACK)
        winner_rect = winner_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT - 60))
        self.screen.blit(winner_text, winner_rect)

    def run(self):
        running = True
        while running:
            mouse_pos = pygame.mouse.get_pos()
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if self.game_state == "menu":
                        if self.berserker_vs_tactician_btn.is_clicked(mouse_pos):
                            self.start_game(AIPersonality.BERSERKER, AIPersonality.TACTICIAN)
                        elif self.tactician_vs_opportunist_btn.is_clicked(mouse_pos):
                            self.start_game(AIPersonality.TACTICIAN, AIPersonality.OPPORTUNIST)
                        elif self.berserker_vs_opportunist_btn.is_clicked(mouse_pos):
                            self.start_game(AIPersonality.BERSERKER, AIPersonality.OPPORTUNIST)
                    
                    elif self.game_state == "playing":
                        if self.pause_btn.is_clicked(mouse_pos):
                            self.paused = not self.paused
                        elif self.backward_btn.is_clicked(mouse_pos):
                            self.go_backward()
                        elif self.forward_btn.is_clicked(mouse_pos):
                            self.go_forward()
                        elif self.restart_btn.is_clicked(mouse_pos):
                            self.game_state = "menu"
                        
                        if self.paused:
                            if self.resume_btn.is_clicked(mouse_pos):
                                self.paused = False
                            elif self.menu_btn.is_clicked(mouse_pos):
                                self.game_state = "menu"
                
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        if self.game_state == "playing":
                            self.paused = not self.paused
                        elif self.paused:
                            self.paused = False
                    elif event.key == pygame.K_SPACE:
                        if self.game_state == "playing":
                            self.paused = not self.paused
                    elif event.key == pygame.K_LEFT:
                        if self.game_state == "playing":
                            self.go_backward()
                    elif event.key == pygame.K_RIGHT:
                        if self.game_state == "playing":
                            self.go_forward()
            
            if self.game_state == "menu":
                self.berserker_vs_tactician_btn.update_hover(mouse_pos)
                self.tactician_vs_opportunist_btn.update_hover(mouse_pos)
                self.berserker_vs_opportunist_btn.update_hover(mouse_pos)
            elif self.game_state == "playing":
                self.pause_btn.update_hover(mouse_pos)
                self.backward_btn.update_hover(mouse_pos)
                self.forward_btn.update_hover(mouse_pos)
                self.restart_btn.update_hover(mouse_pos)
                if self.paused:
                    self.resume_btn.update_hover(mouse_pos)
                    self.menu_btn.update_hover(mouse_pos)
            
            self.screen.fill(WHITE)
            
            if self.game_state == "menu":
                self.draw_menu()
            
            elif self.game_state == "playing":
                self.draw_board()
                self.draw_orbs()
                self.draw_players()
                self.draw_info_panel()
                
                self.pause_btn.draw(self.screen, self.small_font)
                self.backward_btn.draw(self.screen, self.small_font)
                self.forward_btn.draw(self.screen, self.small_font)
                self.restart_btn.draw(self.screen, self.small_font)
                
                if self.winner:
                    self.draw_winner()
                
                if self.paused:
                    self.draw_pause_overlay()
                
                if not self.paused and not self.winner:
                    self.process_turn()
            
            pygame.display.flip()
            self.clock.tick(60)
        
        pygame.quit()

if __name__ == "__main__":
    game = MindDuelGame()
    game.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
